# Liberary


In [ ]:
import sqlite3
import requests
import pandas as pd
from sqlalchemy import create_engine

# Configuration

In [38]:
API_KEY = "71c1fe485e704a65a0b103919251411"
CITIES = ["Yerevan", "Moscow", "Tbilisi", "New York", "London"]
DB_NAME = "/Users/hayriyan/Desktop/Code/AI 2.2/Test/Data/DB/MyDB1.db"

engine = create_engine(f"sqlite:///{DB_NAME}")

# Create database & tables


In [39]:
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

# Fetch weather function

In [40]:

def fetch_weather(city_name):
    url = f"http://api.weatherapi.com/v1/current.json?key={API_KEY}&q={city_name}&aqi=no"
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"[ERROR] Failed to fetch {city_name}: {response.text}")
            return None

        data = response.json()
        return {
            "name": data["location"]["name"],
            "country": data["location"]["country"],
            "latitude": data["location"]["lat"],
            "longitude": data["location"]["lon"],
            "temperature": data["current"]["temp_c"],
            "humidity": data["current"]["humidity"],
            "description": data["current"]["condition"]["text"]
        }

    except Exception as e:
        print(f"[EXCEPTION] Error fetching {city_name}: {e}")
        return None



# Insert into DB with logging


In [41]:

city_rows = []
weather_rows = []

for city in CITIES:
    info = fetch_weather(city)
    if info is None:
        continue

    city_rows.append({
        "name": info["name"],
        "country": info["country"],
        "latitude": info["latitude"],
        "longitude": info["longitude"]
    })

    weather_rows.append({
        "city_name": info["name"],
        "temperature": info["temperature"],
        "humidity": info["humidity"],
        "description": info["description"]
    })



In [42]:

df_cities = pd.DataFrame(city_rows)
df_weather = pd.DataFrame(weather_rows)

print("\n=== Cities DataFrame ===")
print(df_cities)

print("\n=== Weather DataFrame ===")
print(df_weather)



=== Cities DataFrame ===
       name                   country  latitude  longitude
0   Yerevan                   Armenia   40.1811    44.5136
1    Moscow                    Russia   55.7522    37.6156
2   Tbilisi                   Georgia   41.7250    44.7908
3  New York  United States of America   40.7142   -74.0064
4    London            United Kingdom   51.5171    -0.1062

=== Weather DataFrame ===
  city_name  temperature  humidity    description
0   Yerevan         10.2        66           Mist
1    Moscow         -2.7        93  Partly cloudy
2   Tbilisi         12.3        71       Overcast
3  New York         12.2        40  Partly Cloudy
4    London          9.0        66  Partly cloudy


In [ ]:

df_cities.to_sql("Cities", con=engine, if_exists="append", index=False)

print("\n[SUCCESS] Cities inserted into database!")



[SUCCESS] Cities inserted into database!


In [45]:
df = pd.read_sql_query("SELECT * FROM Cities;", conn)
conn.close()

df

,name,country,latitude,longitude
0,Yerevan,Armenia,40.1811,44.5136
1,Moscow,Russia,55.7522,37.6156
2,Tbilisi,Georgia,41.7250,44.7908
3,New York,United States of America,40.7142,-74.0064
4,London,United Kingdom,51.5171,-0.1062
5,Yerevan,Armenia,40.1811,44.5136
6,Moscow,Russia,55.7522,37.6156
7,Tbilisi,Georgia,41.7250,44.7908
8,New York,United States of America,40.7142,-74.0064
9,London,United Kingdom,51.5171,-0.1062
